In [1]:
import pymongo as mg
import pandas as pd 

In [2]:
client = mg.MongoClient(host='mongodb://localhost:27017')

In [3]:
database = client['db_medicals'] #Database 선택

In [4]:
collection = database['RecurrenceOfSurgery']

In [5]:
cursor = collection.find({})
data_list = list(cursor)

In [6]:
df_ROS = pd.DataFrame(data_list)
df_ROS.head(2)

,_id,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,64d33b0c6bf1d3b9fa700af8,0,1PT,22.8,3,51.0,0.0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,64d33b0c6bf1d3b9fa700af9,1,2PT,44.9,4,26.0,0.0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [8]:
df_ROS.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _id                     1894 non-null   object 
 1   Unnamed: 0              1894 non-null   int64  
 2   환자ID                    1894 non-null   object 
 3   Large Lymphocyte        1894 non-null   float64
 4   Location of herniation  1894 non-null   int64  
 5   ODI                     462 non-null    float64
 6   가족력                     1843 non-null   float64
 7   간질성폐질환                  1894 non-null   int64  
 8   고혈압여부                   1894 non-null   int64  
 9   과거수술횟수                  1894 non-null   int64  
 10  당뇨여부                    1894 non-null   int64  
 11  말초동맥질환여부                1894 non-null   int64  
 12  빈혈여부                    1894 non-null   int64  
 13  성별                      1894 non-null   int64  
 14  스테로이드치료                 1894 non-null   

## 분석기준 : 연령대에 따라 발생되는 질병의 종류 분포도

### 불필요한 columns drop

In [9]:
df_ROS.columns  

Index(['_id', 'Unnamed: 0', '환자ID', 'Large Lymphocyte',
       'Location of herniation', 'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수',
       '당뇨여부', '말초동맥질환여부', '빈혈여부', '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환',
       '암발병여부', '연령', '우울증여부', '입원기간', '입원일자', '종양진행여부', '직업', '체중', '퇴원일자',
       '헤모글로빈수치', '혈전합병증여부', '환자통증정도', '흡연여부', '통증기간(월)', '수술기법', '수술시간',
       '수술실패여부', '수술일자', '재발여부', '혈액형', '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도',
       'Instability', 'MF + ES', 'Modic change', 'PI', 'PT', 'Seg Angle(raw)',
       'Vaccum disc', '골밀도', '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [29]:
df_ROS_drops = df_ROS.drop(columns=['_id','Unnamed: 0','Large Lymphocyte',
       'Location of herniation', 'ODI', '가족력','간질성폐질환','과거수술횟수','성별', '스테로이드치료','신장','입원기간', '입원일자', '직업', '체중', '퇴원일자',
       '헤모글로빈수치','환자통증정도', '흡연여부', '통증기간(월)', '수술기법','수술시간',
       '수술실패여부', '수술일자', '재발여부', '혈액형', '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도','Instability', 'MF + ES', 'Modic change', 'PI', 'PT', 'Seg Angle(raw)',
       'Vaccum disc', '골밀도', '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증' ])
df_ROS_drops.head(2)

,환자ID,고혈압여부,당뇨여부,말초동맥질환여부,빈혈여부,신부전여부,심혈관질환,암발병여부,연령,우울증여부,종양진행여부,혈전합병증여부
0,1PT,0,0,0,0,0,0,0,66,0,0,0
1,2PT,0,0,0,0,0,0,0,47,0,0,0


### 데이터 타입 확인 

In [12]:
df_ROS_drops.describe() # 수치형

,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,말초동맥질환여부,빈혈여부,...,MF + ES,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추전방위증
count,1894.000000,1894.000000,462.000000,1843.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,...,1894.000000,1894.000000,1890.000000,1890.000000,1893.000000,1894.000000,998.000000,1893.000000,1894.000000,1894.000000
mean,39.270750,2.295671,31.142857,0.049919,0.020063,0.130940,0.302534,0.062830,0.004224,0.024287,...,2096.421600,0.247096,46.593651,16.329841,15.287934,0.056494,-1.507325,1989.788774,4.430834,0.032207
std,13.675874,0.996143,10.131304,0.217836,0.140254,0.337424,0.638461,0.242721,0.064871,0.153980,...,472.834488,0.661506,22.736211,8.421322,10.426397,0.230935,0.495867,314.478436,3.595680,0.176596
min,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1016.800000,0.000000,11.900000,-10.600000,-27.400000,0.000000,-2.840000,1145.800000,1.000000,0.000000
25%,30.700000,1.000000,24.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1765.285000,0.000000,38.700000,10.500000,8.800000,0.000000,-1.820000,1777.000000,4.000000,0.000000
50%,40.200000,2.000000,31.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2056.500000,0.000000,45.100000,15.700000,14.500000,0.000000,-1.520000,1985.500000,4.000000,0.000000
75%,48.600000,3.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2374.350000,0.000000,52.000000,21.400000,21.100000,0.000000,-1.202500,2178.500000,4.000000,0.000000
max,89.000000,5.000000,68.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,...,5569.000000,3.000000,559.000000,48.700000,165.000000,1.000000,1.700000,3214.200000,45.000000,1.000000


In [13]:
df_ROS_drops.describe(include=object) # 범주형

,환자ID,직업,수술기법,혈액형,척추이동척도
count,1894,1479,1813,1894,1894
unique,1894,17,2,4,5
top,1PT,사무직,TELD,RH+A,Middle
freq,1,449,1673,688,1260


In [27]:
df_ROS_drops['당뇨여부'].value_counts()

0    1775
1     119
Name: 당뇨여부, dtype: int64

In [30]:
df_ROS_drops['당뇨여부'].unique()

array([0, 1], dtype=int64)